In [1]:
import pandas as pd
import numpy as np

In [2]:
# IMPORTAÇÃO DA BASE DE EMPRESAS PUNIDAS
colunas = ['CPF OU CNPJ DO SANCIONADO', 'NOME INFORMADO PELO ÓRGÃO SANCIONADOR',
           'DATA INÍCIO SANÇÃO', 'DATA FINAL SANÇÃO']
punidas_df = pd.read_csv('C:\\DATASETS\Empresas Punidas\\1-Arquivos Baixados\\20210408_CNEP.csv',
                         sep=';', dtype='str', usecols=colunas, encoding='latin-1')

Processamento/Tratamento de Dados:

In [3]:
# Conferindo campos e tipos
punidas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 4 columns):
CPF OU CNPJ DO SANCIONADO                297 non-null object
NOME INFORMADO PELO ÓRGÃO SANCIONADOR    297 non-null object
DATA INÍCIO SANÇÃO                       297 non-null object
DATA FINAL SANÇÃO                        21 non-null object
dtypes: object(4)
memory usage: 9.4+ KB


In [4]:
# Conferindo campos e valores
punidas_df.describe

<bound method NDFrame.describe of     CPF OU CNPJ DO SANCIONADO  \
0                 34058422807   
1              21504073000190   
2              10699386000125   
3              07486627000143   
4              21463538000102   
..                        ...   
292               35735286862   
293            07487504000127   
294            07487504000127   
295            08143105000101   
296            13197596000103   

                 NOME INFORMADO PELO ÓRGÃO SANCIONADOR DATA INÍCIO SANÇÃO  \
0                                SEVERINO JOSÉ MAFALDO         26/12/2018   
1    CENTRO DE FORMAÇÃO DE CONDUTORES SERRACAR LARA...         19/09/2018   
2    TEC LIGHT INDUSTRIA E COMERCIO DE REATORES EIRELI         22/11/2019   
3                                   C.A.M. CASTILHO ME         14/12/2020   
4                        COMERCIAL PICAPAU EIRELI - ME         17/03/2020   
..                                                 ...                ...   
292                     RONALD

In [5]:
# Identificando percentual de valores ausentes por coluna:
(punidas_df.isnull().sum() / punidas_df.shape[0] * 100)
     .sort_values(ascending=False)

DATA FINAL SANÇÃO                        92.929293
DATA INÍCIO SANÇÃO                        0.000000
NOME INFORMADO PELO ÓRGÃO SANCIONADOR     0.000000
CPF OU CNPJ DO SANCIONADO                 0.000000
dtype: float64

### Limpeza e correção dos dados:

##### Limpeza e correção da coluna 'CPF OU CNPJ DO SANCIONADO':

In [6]:
# Conferindo se a coluna 'CPF OU CNPJ DO SANCIONADO' contém realmente
# apenas CPF(11 dígitos) ou CNPJ(14 dígitos). Também fornece informação 
# sobre quantas pessoas físicas e quantas pessoas jurídicas estão presentes na base:
punidas_df['CPF OU CNPJ DO SANCIONADO'].str.len().value_counts()

14    272
11     25
Name: CPF OU CNPJ DO SANCIONADO, dtype: int64

Não há o que corrigir / limpar.

### Análise dos dados:

In [7]:
punidas_df

,CPF OU CNPJ DO SANCIONADO,NOME INFORMADO PELO ÓRGÃO SANCIONADOR,DATA INÍCIO SANÇÃO,DATA FINAL SANÇÃO
0,34058422807,SEVERINO JOSÉ MAFALDO,26/12/2018,NaN
1,21504073000190,CENTRO DE FORMAÇÃO DE CONDUTORES SERRACAR LARA...,19/09/2018,NaN
2,10699386000125,TEC LIGHT INDUSTRIA E COMERCIO DE REATORES EIRELI,22/11/2019,NaN
3,07486627000143,C.A.M. CASTILHO ME,14/12/2020,NaN
4,21463538000102,COMERCIAL PICAPAU EIRELI - ME,17/03/2020,NaN
...,...,...,...,...
292,35735286862,RONALDO VIANA MARTINS CARVALHO,28/01/2019,NaN
293,07487504000127,MOURA E MOURA INFORMÁTICA E EMPREENDIMENTOS LTDA,04/02/2021,NaN
294,07487504000127,MOURA E MOURA INFORMATICA E EMPREENDIMENTO LTDA,04/06/2019,NaN
295,08143105000101,COMERCIO DE MATERIAIS PARA CONSTRUÇÃO MARIGU L...,12/09/2019,NaN


In [9]:
print(punidas_df['DATA INÍCIO SANÇÃO'].str[-4:].min())
print(punidas_df['DATA INÍCIO SANÇÃO'].str[-4:].max())

2015
2021


Datas de início dentro do esperado.

In [10]:
print(punidas_df[punidas_df['DATA FINAL SANÇÃO'].notnull() == True]['DATA FINAL SANÇÃO'].str[-4:].min())
print(punidas_df[punidas_df['DATA FINAL SANÇÃO'].notnull() == True]['DATA FINAL SANÇÃO'].str[-4:].max())

2021
2025


Já sobre a data final, percebemos um registro que pode ser um erro (31/12/2099). Mas  Além disso, vimos anteriormente que há linhas com 'DATA FINAL SANÇÃO', e também não faremos tratamento sobre isso.

Como iremos considerar apenas a existência ou não da empresa na lista de punidas, então não é preciso nenhum tratamento adicional. Vamos remover as colunas de datas, pois serviram apenas para conhecimento e não serão mais necessárias:

In [11]:
punidas_df.drop(columns=['DATA INÍCIO SANÇÃO', 'DATA FINAL SANÇÃO'], inplace=True)

Vamos agora remover duplicidades:

In [13]:
# Vamos ver se existem duplicidades de cnpj ou cpf:
punidas_df['CPF OU CNPJ DO SANCIONADO'].value_counts()

07487504000127    7
34058422807       6
35735286862       6
21891781000121    3
04977561000104    2
                 ..
20520956000121    1
13540305000138    1
03821415000114    1
03248412000486    1
15329604000153    1
Name: CPF OU CNPJ DO SANCIONADO, Length: 203, dtype: int64

In [14]:
# Vamos separar um dataframe só com CNPJ:
punidas_pj_df = punidas_df[punidas_df['CPF OU CNPJ DO SANCIONADO'].str.len()==14]

In [15]:
# Vamos agrupar por CNPJ e pegar apenas o primeiro nome da empresa:
punidas_pj_df = punidas_pj_df.groupby('CPF OU CNPJ DO SANCIONADO').head(1)

In [16]:
# Escolhido o cnpj 07487504000127, vamos ver como ele estava na lista original:
punidas_df[punidas_df['CPF OU CNPJ DO SANCIONADO']=='07487504000127']

,CPF OU CNPJ DO SANCIONADO,NOME INFORMADO PELO ÓRGÃO SANCIONADOR
95,07487504000127,MOURA E MOURA INFORMATICA E EMPREENDIMENTOS LTDA
181,07487504000127,MOURA E MOURA INFORMÁTICA E EMPREENDIMENTOS CO...
236,07487504000127,MOURA E MOURA INFORMATICA E EMPREENDIMENTOS LTDA
267,07487504000127,MOURA E MOURA INFORMATICA E EMPREENDIMENTOS LTDA
281,07487504000127,MOURA E MOURA INFORMATICA E EMPREENDIMENTOS CO...
293,07487504000127,MOURA E MOURA INFORMÁTICA E EMPREENDIMENTOS LTDA
294,07487504000127,MOURA E MOURA INFORMATICA E EMPREENDIMENTO LTDA


In [17]:
# Vamos ver como esse mesmo CNPJ ficou na lista consolidada (sem duplicidade):
punidas_pj_df[punidas_pj_df['CPF OU CNPJ DO SANCIONADO']=='07487504000127']

,CPF OU CNPJ DO SANCIONADO,NOME INFORMADO PELO ÓRGÃO SANCIONADOR
95,07487504000127,MOURA E MOURA INFORMATICA E EMPREENDIMENTOS LTDA


Ou seja, tudo correto na remoção de cnpjs duplicados.

Vamos agora avaliar e remover duplicidades de CPF:

In [18]:
# Primeiro, separar um dataframe só com CPF:
punidas_pf_df = punidas_df[punidas_df['CPF OU CNPJ DO SANCIONADO'].str.len()==11]

In [19]:
# Como não podemos desconsiderar o nome nos casos de linhas de CPF, precisamos remover duplicados usando o método drop, considerando as duas colunas:
print(punidas_pf_df.shape)
punidas_pf_df = punidas_pf_df.drop_duplicates()
print(punidas_pf_df.shape)

(25, 2)
(11, 2)


Removidas as duplicidades de cpf e cnpj, vamos unificar os dataframes:

In [20]:
punidas_df = pd.concat([punidas_pj_df, punidas_pf_df])
punidas_df.reset_index(drop=True)

,CPF OU CNPJ DO SANCIONADO,NOME INFORMADO PELO ÓRGÃO SANCIONADOR
0,21504073000190,CENTRO DE FORMAÇÃO DE CONDUTORES SERRACAR LARA...
1,10699386000125,TEC LIGHT INDUSTRIA E COMERCIO DE REATORES EIRELI
2,07486627000143,C.A.M. CASTILHO ME
3,21463538000102,COMERCIAL PICAPAU EIRELI - ME
4,06940189000189,MEC PRESS ENGENHARIA DO BRASIL EIRELI
...,...,...
198,25980213805,RODRIGO ZANARDO
199,30872074897,BRUNO SOARES BERNARDO
200,14403279805,VIVALDO DIAS DA SILVA
201,03133444930,CARLOS ARTURO MALLORQUIN JUNIOR


Verificamos acima alguns nomes com acentuação. Vamos retirar os acentos e colocar tudo em maiúsculo, pois a base de sócios está dessa forma (e precisaremos fazer join de pessoas físicas usando a coluna de nome).

In [22]:
# Definição de uma função para retirar acentos e transformar texto em maiúsculo:

import unicodedata

def strip_accents_and_upper(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text).upper()

s = strip_accents_and_upper('àéêöhello')
print(s)

AEEOHELLO


In [23]:
# Aplicação da função
punidas_df['NOME INFORMADO PELO ÓRGÃO SANCIONADOR'] = punidas_df['NOME INFORMADO PELO ÓRGÃO SANCIONADOR'].map(strip_accents_and_upper)

In [24]:
# Verificando resultados:
punidas_df

,CPF OU CNPJ DO SANCIONADO,NOME INFORMADO PELO ÓRGÃO SANCIONADOR
1,21504073000190,CENTRO DE FORMACAO DE CONDUTORES SERRACAR LARA...
2,10699386000125,TEC LIGHT INDUSTRIA E COMERCIO DE REATORES EIRELI
3,07486627000143,C.A.M. CASTILHO ME
4,21463538000102,COMERCIAL PICAPAU EIRELI - ME
5,06940189000189,MEC PRESS ENGENHARIA DO BRASIL EIRELI
...,...,...
68,25980213805,RODRIGO ZANARDO
78,30872074897,BRUNO SOARES BERNARDO
111,14403279805,VIVALDO DIAS DA SILVA
126,03133444930,CARLOS ARTURO MALLORQUIN JUNIOR


Por fim, vamos renomear colunas:

In [25]:
punidas_df = punidas_df.rename(columns={"CPF OU CNPJ DO SANCIONADO": "CPF_CNPJ", "NOME INFORMADO PELO ÓRGÃO SANCIONADOR": "NOME"})

Finalizado. Vamos salvar os resultados:

In [26]:
punidas_df.to_csv(r'C:\\DATASETS\Empresas Punidas\\punidas.csv', index = False)